# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer


import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_database.db')

#Read data from the table we created earlier and we call it 'disaster_response'
df = pd.read_sql_table('disaster_response', engine)

# the X is the feature we are going to use to train and test our ML model
X = df.filter(items=['id', 'message', 'original', 'genre'])

# the y contain result of the calssification we used to to classify each message 
y = df.drop(['id', 'message', 'original', 'genre', 'child_alone', 'related'], axis=1)

#'child_alone' has no responses thats why we dropped it
#'related' columns has been dropped since it contains non binary numbers 2

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### 2. Write a tokenization function to process your text data

In [3]:
# This is tokenize function we are goining to use to tokenize word
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Create a pipeline instance which contian the process of fitting and transformnig our data 
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(LinearSVC(multi_class="crammer_singer"), n_jobs=1))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# use the train_split to split our data to training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...rammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Print out Precision , recall F1_score and support for each column using classification_report function
y_pred_test = pipeline.predict(X_test['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))


                        precision    recall  f1-score   support

               request       0.77      0.63      0.69      1108
                 offer       0.00      0.00      0.00        23
           aid_related       0.71      0.71      0.71      2686
          medical_help       0.61      0.29      0.40       548
      medical_products       0.71      0.35      0.47       317
     search_and_rescue       0.67      0.20      0.31       177
              security       0.67      0.02      0.03       129
              military       0.58      0.30      0.39       203
                 water       0.76      0.67      0.71       419
                  food       0.81      0.74      0.77       725
               shelter       0.75      0.61      0.68       558
              clothing       0.79      0.56      0.65        97
                 money       0.56      0.25      0.35       157
        missing_people       0.60      0.18      0.28        84
              refugees       0.64      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# introduce the parameters to enhance our model 
parameters = {
        'clf__estimator__C': [1, 1.2, 1.4],
        'clf__estimator__max_iter': [1000, 1200, 1500],
    } 


In [9]:
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train['message'], y_train)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...rammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__C': [1, 1.2, 1.4], 'clf__estimator__max_iter': [1000, 1200, 1500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
cv.best_params_

{'clf__estimator__C': 1, 'clf__estimator__max_iter': 1000}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred_test = pipeline.predict(X_test['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))


                        precision    recall  f1-score   support

               request       0.77      0.63      0.69      1108
                 offer       0.00      0.00      0.00        23
           aid_related       0.71      0.71      0.71      2686
          medical_help       0.61      0.29      0.40       548
      medical_products       0.71      0.35      0.47       317
     search_and_rescue       0.67      0.20      0.31       177
              security       0.67      0.02      0.03       129
              military       0.58      0.30      0.39       203
                 water       0.76      0.67      0.71       419
                  food       0.81      0.74      0.77       725
               shelter       0.75      0.61      0.68       558
              clothing       0.79      0.56      0.65        97
                 money       0.56      0.25      0.35       157
        missing_people       0.60      0.18      0.28        84
              refugees       0.64      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
randomForest_pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])


In [12]:
randomForest_pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
y_pred_test = randomForest_pipeline.predict(X_test['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))


                        precision    recall  f1-score   support

               request       0.82      0.39      0.53      1108
                 offer       0.00      0.00      0.00        23
           aid_related       0.74      0.52      0.61      2686
          medical_help       0.53      0.05      0.10       548
      medical_products       0.67      0.09      0.16       317
     search_and_rescue       0.64      0.05      0.09       177
              security       0.00      0.00      0.00       129
              military       0.62      0.09      0.16       203
                 water       0.86      0.24      0.37       419
                  food       0.85      0.36      0.50       725
               shelter       0.85      0.29      0.43       558
              clothing       0.73      0.08      0.15        97
                 money       0.71      0.03      0.06       157
        missing_people       0.00      0.00      0.00        84
              refugees       0.71      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [14]:
import pickle
filename = 'disaster_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.